<a href="https://colab.research.google.com/github/Ali-mohammadi-design/Prompt_Engineering_and_Machine_Learning/blob/main/Prompt_engineering_Lang_chain_EBD_TRIZ_Question_ASker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this practice we want to recieve a sentence and ask questions regarding that sentence to gather information

In [1]:
!pip install langchain
!pip install openai
!pip install wikipedia
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requ

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [4]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [137]:
llm= ChatOpenAI(openai_api_key=openAI_API_Key,temperature=0)
llm2= ChatOpenAI(openai_api_key=openAI_API_Key)

In [168]:
system_template1=" You are an AI assistant that gets text and return nouns. Please recognize verbs but do not return verbs or propositions"
system_message_prompt1= SystemMessagePromptTemplate.from_template(system_template1)
human_template1='please list all the nouns in the following text: \n {text}'
human_message_prompt1=HumanMessagePromptTemplate.from_template(human_template1)
prompt1=ChatPromptTemplate.from_messages([system_message_prompt1,human_message_prompt1])
chain1=LLMChain(llm=llm2, prompt=prompt1, output_key='nouns')

In [147]:
human_template2='Consider the following list of nouns. please specify which nouns are human and which nouns are non-human.These are the nouns: {nouns}'
prompt2=ChatPromptTemplate.from_template(human_template2)
chain2=LLMChain(llm=llm, prompt=prompt2, output_key='type')

In [173]:
system_template3=" you are an AI assistant that get nouns and  their type and ask question with what or who. for the human nouns you ask: who is + noun? \n For non-human nouns you ask: what is + noun?"
system_message_prompt3= SystemMessagePromptTemplate.from_template(system_template3)
human_template3='''please consider the folowing nouns. for each noun you must ask questions with what or who depends on their type.
for example if the nouns are teacher and sandwich you shoud return who is teacher? and what is sandwich? The nouns are : \n {type}'''
human_message_prompt3=HumanMessagePromptTemplate.from_template(human_template3)
noun1='Bob, school, sandwich'
example_one= HumanMessagePromptTemplate.from_template(noun1)
question1='who is Bob? \n what is school? \n what is sandwich?'
example_one_output= AIMessagePromptTemplate.from_template(question1)
prompt3=ChatPromptTemplate.from_messages([system_message_prompt3,human_message_prompt3])
chain3=LLMChain(llm=llm, prompt=prompt3, output_key='questions')

In [174]:
text='a cafe that people could study there'
seq_chain=SequentialChain(chains=[chain1,chain2,chain3], input_variables=['text'],output_variables=['nouns','type','questions'],verbose=True)

In [175]:
result=seq_chain(text)



> Entering new SequentialChain chain...

> Finished chain.


In [176]:
result

{'text': 'a cafe that people could study there',
 'nouns': 'cafe, people, study',
 'type': 'Cafe - non-human\nPeople - human\nStudy - non-human',
 'questions': 'What is cafe?\nWho is people?\nWhat is study?'}

In [180]:
system_template0=''' "You are an AI assistant who aims to help designers to extract as much information as possible from a design problem.
You get a design problem as a sentence and ask many questions.
'''
system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template0)
human_template0='''' "I'll provide a sentence. Please examine the given sentence, first identifying its verbs.
After that, take note of all the verbs you've identified and generate questions using 'where, why, who, how, when' based on those verbs.
The sentence is: {text}"
'''
human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template0)
prompt0=ChatPromptTemplate.from_messages([system_template0,human_message_prompt0])
request=prompt0.format_prompt(text=text).to_messages()
result=llm(request)
print(result.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Verbs identified in the sentence: could, study

Questions generated based on the verbs:
1. Where could people study?
2. Why would people study at the cafe?
3. Who could study at the cafe?
4. How could people study at the cafe?
5. When could people study at the cafe?


**As a function to get the sentence and ask question**

In [181]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [182]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [183]:
llm= ChatOpenAI(openai_api_key=openAI_API_Key,temperature=0)

In [216]:
def question_asking(sentence):
  system_template1=" You are an AI assistant that gets text and return nouns. Please recognize verbs but do not return verbs or propositions"
  system_message_prompt1= SystemMessagePromptTemplate.from_template(system_template1)
  human_template1='please list all the nouns in the following text: \n {text}'
  human_message_prompt1=HumanMessagePromptTemplate.from_template(human_template1)
  prompt1=ChatPromptTemplate.from_messages([system_message_prompt1,human_message_prompt1])
  chain1=LLMChain(llm=llm, prompt=prompt1, output_key='nouns')

  human_template2='Consider the following list of nouns. please specify which nouns are human and which nouns are non-human.These are the nouns: {nouns}'
  prompt2=ChatPromptTemplate.from_template(human_template2)
  chain2=LLMChain(llm=llm, prompt=prompt2, output_key='type')

  system_template3=" you are an AI assistant that get nouns and  their type and ask question with what or who. for the human nouns you ask: who is + noun? \n For non-human nouns you ask: what is + noun?"
  system_message_prompt3= SystemMessagePromptTemplate.from_template(system_template3)
  human_template3='''please consider the folowing nouns. for each noun you must ask questions with what or who depends on their type.
  for example if the nouns are teacher and sandwich you shoud return who is teacher? and what is sandwich? The nouns are : \n {type}'''
  human_message_prompt3=HumanMessagePromptTemplate.from_template(human_template3)
  noun1='Bob, school, sandwich'
  example_one= HumanMessagePromptTemplate.from_template(noun1)
  question1='who is Bob? \n what is school? \n what is sandwich?'
  example_one_output= AIMessagePromptTemplate.from_template(question1)
  prompt3=ChatPromptTemplate.from_messages([system_message_prompt3,human_message_prompt3])
  chain3=LLMChain(llm=llm, prompt=prompt3, output_key='questions')

  text=sentence
  seq_chain=SequentialChain(chains=[chain1,chain2,chain3], input_variables=['text'],output_variables=['nouns','type','questions'],verbose=True)
  result1=seq_chain(text)
  noun_questions=result1


  system_template0=''' "You are an AI assistant who aims to help designers to extract as much information as possible from a design problem.
  You get a design problem as a sentence and ask questions from each verb that you recognize in the sentence.
  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template0)
  human_template0='''' "I'll provide a sentence for you. Please examine the given sentence, first identifying its verbs.
  After that, take note of each verb you've identified in the provided sentence and generate questions for each verb using 'where, why, who, how, when' based on those verbs.
  \n The sentence is: \n {text}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template0)
  prompt0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  request=prompt0.format_prompt(text=text).to_messages()
  result0=llm(request)
  verb_questions=result0

  return print(f"the noun questions are: \n{noun_questions['questions']} \n\n\n the verb questions are:\n \n{verb_questions.content}")

In [217]:
question_asking('design a house that can fly easily from one loaction to another location')



> Entering new SequentialChain chain...

> Finished chain.
the noun questions are: 
What is house?
Who is location? 


 the verb questions are:
 
Verbs identified in the sentence: design, fly

Questions for the verb 'design':
1. Where should the house be designed?
2. Why is the house being designed?
3. Who is responsible for designing the house?
4. How should the house be designed?
5. When should the house be designed?

Questions for the verb 'fly':
1. Where should the house be able to fly?
2. Why should the house be able to fly?
3. Who will operate the house while flying?
4. How will the house fly?
5. When will the house need to fly?
